# Metaprogramming

From Wikipedia:

> In computer programming, **homoiconicity** (from the Greek words homo- meaning "the same" and icon meaning "representation") is a property of some programming languages. A language is **homoiconic** if a program written in it can be manipulated as data using the language, and thus the program's internal representation can be inferred just by reading the program itself. For example, a Lisp program is written as a regular Lisp list, and can be manipulated by other Lisp code.[1] In homoiconic languages, all code can be accessed and transformed as data, using the same representation. This property is often summarized by saying that the language treats "code as data".

Julia is homoiconic. In Julia, program code can be represented by a Julia data structure called an expression.

In [ ]:
Meta.parse("2 + 3")

In [ ]:
typeof(ans)

In [ ]:
two_plus_three = :(2 + 3)

This is known as *quoting* and `:` is the `quote` operator.

Code with more than one line can be quoted like this:

In [ ]:
quote
    a = 42
    b = a^2
    a - b
end

In [ ]:
eval(two_plus_three)

We can use the `dump` function to see how any value in Julia is represented.

In [ ]:
dump(two_plus_three)

`head` indicates that this expression is a function call. `args` is an array containing the function and its arguments.

In [ ]:
two_plus_three.args[1]

Let's make a copy of the expression and modify the first `arg`

In [ ]:
two_minus_three = copy(two_plus_three)
two_minus_three.args[1] = :-

In [ ]:
two_plus_three

Now we can look at and evaluate the modified expression.

In [ ]:
two_minus_three

In [ ]:
eval(two_minus_three)

## Macro example

Using the above and just a little more metaprogramming machinery, we can create powerful *macros*.

Here is an example.

In [ ]:
macro timeit(ex)
    quote
        local t0 = time()
        local val = $(esc(ex))
        local t1 = time()
        println("elapsed time: ", t1-t0, " seconds")
        val
    end
end

In [ ]:
@timeit factorial(20)

Note: Julia comes with a built-in `@time` macro.

In [ ]:
@time factorial(20)

# Taylor series example

Based on an example by Mike J Innes.

## Original sin

Here's a more practical example. Consider the following definition of the `sin` function, based on the [Taylor series](https://en.wikipedia.org/wiki/Taylor_series).

$$sin(x) = \sum_{k=0}^{\infty} \frac{(-1)^k}{(1+2k)!} x^{1+2k}$$

or

$$sin(x) = x - \frac{x^3}{3!} + \frac{x^5}{5!} - \frac{x^7}{7!} ...$$

*Aside:* The following code uses a generator expression, which is a comprehension written without the square brackets. Generator expressions produce values on demand without storing them.

In [ ]:
mysin(x) = sum((-1)^k/factorial(1+2k) * x^(1+2k) for k = 0:9)

In [ ]:
mysin(0.5), sin(0.5)

To see where we are right now, we'll benchmark it.

In [ ]:
using BenchmarkTools
@benchmark mysin(0.5)

Right now, this is much slower than it could be. The reason is that we're looping over `k`, which is relatively expensive. It'd be much faster to write out:

In [ ]:
mysin(x) = x - x^3/6 + x^5/120 + x^7/5040 # + ...

But this is tedious to write, and no longer looks like the original Taylor series. It's harder to tell if we've made a mistake, and we easily modify it. Is there a way to get the best of both worlds?

How about getting Julia to write out that code for us?

To start with, let's consider a symbolic version of the `+` function.

In [ ]:
plus(a, b) = :($a + $b)

This is a function that returns an expression.

In [ ]:
plus(1, 2)

With `plus` we can do more interesting things, like symbolic `sum`:

In [ ]:
reduce(+, 1:10)

In [ ]:
reduce(plus, 1:10)

In [ ]:
eval(ans)

Given that, we can also sum over symbolic variables.

In [ ]:
reduce(plus, [:(x^2), :x, 1])

This gives us an important piece of the puzzle, but we also need to figure out _what_ we're summing. Let's crate a symbolic version of the Taylor series above, which interpolates the value of `k`.

In [ ]:
k = 3
:($((-1)^k) * x^$(1+2k) / $(factorial(1+2k)))

Now we have one term, we can generate as many as we like.

In [ ]:
terms = [:($((-1)^k) * x^$(1+2k) / $(factorial(1+2k))) for k = 0:9]

And sum them –

In [ ]:
taylor_series = reduce(plus, terms)

And create a function definition out of it:

In [ ]:
:(mysin(x) = $taylor_series)

In [ ]:
eval(ans)

In [ ]:
mysin(0.5), sin(0.5)

In [ ]:
@benchmark mysin(0.5)

Compare this to the benchmark results above.